# Financial Forecasting with Covariates & Exogenous Variables
## A Practical Guide for JPMorgan-Style Analysis

This notebook demonstrates how to properly incorporate different types of covariates in financial time series forecasting:
- Exogenous variables (macro factors)
- Future-known covariates (calendar features)
- Lagged covariates (historical values)
- Static covariates (unchanging characteristics)

We'll build a stock price forecasting model that combines all these effectively.

## 1. Setup and Data Generation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 6)

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Generate Realistic Financial Dataset

We'll create a synthetic dataset that mimics real financial markets with:
- Stock prices influenced by market conditions
- Macro-economic factors (exogenous)
- Calendar effects (future-known)
- Company characteristics (static)

In [ ]:
def generate_financial_dataset(n_days=1000, start_date='2021-01-01'):
    """
    Generate realistic financial dataset with multiple covariate types
    """
    # Create date range (business days only)
    dates = pd.bdate_range(start=start_date, periods=n_days, freq='B')
    
    df = pd.DataFrame({'date': dates})
    df['date'] = pd.to_datetime(df['date'])
    
    # ============================================
    # FUTURE-KNOWN COVARIATES (Calendar features)
    # ============================================
    df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 4=Friday
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['day_of_month'] = df['date'].dt.day
    df['is_month_end'] = (df['date'].dt.is_month_end).astype(int)
    df['is_quarter_end'] = ((df['date'].dt.month % 3 == 0) & df['is_month_end']).astype(int)
    
    # Earnings dates (scheduled quarterly, known in advance)
    df['is_earnings_day'] = ((df['day_of_month'] >= 20) & 
                             (df['day_of_month'] <= 22) & 
                             df['is_quarter_end']).astype(int)
    
    # ============================================
    # EXOGENOUS VARIABLES (External macro factors)
    # ============================================
    
    # Interest rates (slow-moving, controlled by Fed)
    base_rate = 2.0
    rate_changes = np.cumsum(np.random.normal(0, 0.01, n_days))
    df['interest_rate'] = np.clip(base_rate + rate_changes, 0.5, 5.0)
    
    # Market index (S&P 500 proxy - affects all stocks)
    market_trend = 0.0003 * np.arange(n_days)  # Slow upward trend
    market_volatility = np.random.normal(0, 0.02, n_days)
    market_seasonal = 0.05 * np.sin(2 * np.pi * np.arange(n_days) / 252)  # Yearly cycle
    df['market_index'] = 100 * np.exp(np.cumsum(market_trend + market_volatility + market_seasonal))
    
    # VIX (volatility index - fear gauge)
    base_vix = 15
    vix_shocks = np.random.choice([0, 0, 0, 5, -3], n_days)  # Occasional spikes
    df['vix'] = np.clip(base_vix + np.cumsum(np.random.normal(0, 1, n_days)) + vix_shocks, 10, 50)
    
    # GDP Growth Rate (quarterly data, slow-moving)
    gdp_base = 2.5
    gdp_quarterly = gdp_base + np.random.normal(0, 0.5, n_days // 63 + 1)
    df['gdp_growth'] = np.repeat(gdp_quarterly, 63)[:n_days]
    
    # Unemployment rate (economic health indicator)
    unemployment_base = 5.0
    unemployment_trend = -0.00002 * np.arange(n_days)  # Slowly improving
    df['unemployment_rate'] = np.clip(
        unemployment_base + unemployment_trend + np.cumsum(np.random.normal(0, 0.02, n_days)),
        3.5, 8.0
    )
    
    # Oil prices (affects many sectors)
    oil_base = 60
    oil_volatility = np.random.normal(0, 2, n_days)
    df['oil_price'] = np.clip(oil_base + np.cumsum(oil_volatility), 40, 100)
    
    # ============================================
    # TARGET: STOCK PRICE (influenced by above)
    # ============================================
    
    stock_base = 100
    
    # Company-specific trend
    company_trend = 0.0005 * np.arange(n_days)
    
    # Market correlation (beta effect)
    market_effect = 0.7 * (df['market_index'].pct_change().fillna(0))
    
    # Macro factors influence
    rate_effect = -0.3 * df['interest_rate'].pct_change().fillna(0)  # Inverse relationship
    vix_effect = -0.2 * df['vix'].pct_change().fillna(0)  # High VIX = risk off
    
    # Calendar effects
    monday_effect = -0.002 * (df['day_of_week'] == 0)  # Monday slightly negative
    friday_effect = 0.001 * (df['day_of_week'] == 4)   # Friday slightly positive
    earnings_effect = 0.03 * df['is_earnings_day']      # Earnings day volatility
    month_end_effect = 0.005 * df['is_month_end']       # Month-end rebalancing
    
    # Company-specific volatility
    idiosyncratic = np.random.normal(0, 0.015, n_days)
    
    # Combine all effects
    returns = (company_trend + market_effect + rate_effect + vix_effect +
               monday_effect + friday_effect + earnings_effect + 
               month_end_effect + idiosyncratic)
    
    df['stock_price'] = stock_base * np.exp(np.cumsum(returns))
    
    # ============================================
    # LAGGED COVARIATES (Historical values)
    # ============================================
    
    # Trading volume (influenced by price movements and VIX)
    base_volume = 1000000
    volume_from_volatility = base_volume * (1 + 0.02 * df['vix'])
    volume_noise = np.random.normal(0, 0.1 * base_volume, n_days)
    df['volume'] = np.abs(volume_from_volatility + volume_noise)
    
    # Price momentum indicators
    df['price_return_1d'] = df['stock_price'].pct_change(1).fillna(0)
    df['price_return_5d'] = df['stock_price'].pct_change(5).fillna(0)
    df['price_return_20d'] = df['stock_price'].pct_change(20).fillna(0)
    
    # Moving averages (technical indicators)
    df['sma_5'] = df['stock_price'].rolling(5).mean().fillna(method='bfill')
    df['sma_20'] = df['stock_price'].rolling(20).mean().fillna(method='bfill')
    df['sma_50'] = df['stock_price'].rolling(50).mean().fillna(method='bfill')
    
    # Volatility (20-day rolling std)
    df['realized_volatility'] = df['price_return_1d'].rolling(20).std().fillna(method='bfill')
    
    return df

# Generate dataset
df = generate_financial_dataset(n_days=1000)

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head(10))
print("\nDataset info:")
print(df.info())

## 3. Visualize the Data and Relationships

In [ ]:
# Plot stock price and key covariates
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Stock price
axes[0].plot(df['date'], df['stock_price'], label='Stock Price', color='blue', linewidth=2)
axes[0].set_title('Target: Stock Price', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Exogenous variables
ax1 = axes[1]
ax2 = ax1.twinx()
ax1.plot(df['date'], df['market_index'], label='Market Index', color='green', alpha=0.7)
ax2.plot(df['date'], df['vix'], label='VIX', color='red', alpha=0.7)
ax1.set_ylabel('Market Index', color='green')
ax2.set_ylabel('VIX', color='red')
ax1.set_title('Exogenous Variables: Market Index & VIX', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# More exogenous variables
ax3 = axes[2]
ax4 = ax3.twinx()
ax3.plot(df['date'], df['interest_rate'], label='Interest Rate', color='purple', alpha=0.7)
ax4.plot(df['date'], df['unemployment_rate'], label='Unemployment', color='orange', alpha=0.7)
ax3.set_ylabel('Interest Rate (%)', color='purple')
ax4.set_ylabel('Unemployment (%)', color='orange')
ax3.set_title('Exogenous Variables: Interest Rate & Unemployment', fontsize=12, fontweight='bold')
ax3.legend(loc='upper left')
ax4.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Volume (lagged covariate)
axes[3].bar(df['date'], df['volume'], label='Trading Volume', color='gray', alpha=0.6, width=1)
axes[3].set_title('Lagged Covariate: Trading Volume', fontsize=12, fontweight='bold')
axes[3].set_ylabel('Volume')
axes[3].set_xlabel('Date')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
correlation_vars = ['stock_price', 'market_index', 'vix', 'interest_rate', 
                    'unemployment_rate', 'oil_price', 'volume']

plt.figure(figsize=(10, 8))
sns.heatmap(df[correlation_vars].corr(), annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('Correlation Matrix: Target vs Covariates', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- Stock price is positively correlated with market index")
print("- Negative correlation with VIX (fear index)")
print("- Interest rates show inverse relationship")
print("- Volume increases with volatility (VIX)")

## 4. Define Covariate Categories

Critical step: Properly categorize variables for the model

In [ ]:
# Define covariate categories
covariate_config = {
    # What we're trying to predict
    'target': 'stock_price',
    
    # FUTURE-KNOWN: We know these values for future dates
    'future_known': [
        'day_of_week',
        'month', 
        'quarter',
        'day_of_month',
        'is_month_end',
        'is_quarter_end',
        'is_earnings_day'
    ],
    
    # EXOGENOUS: External factors (we can observe but not predict)
    # In practice, you'd need separate models to forecast these
    # or use external forecasts (e.g., Fed interest rate projections)
    'exogenous': [
        'market_index',
        'vix',
        'interest_rate',
        'gdp_growth',
        'unemployment_rate',
        'oil_price'
    ],
    
    # LAGGED: Historical values and derived features
    # Can only use past values, not future
    'lagged': [
        'volume',
        'price_return_1d',
        'price_return_5d', 
        'price_return_20d',
        'sma_5',
        'sma_20',
        'sma_50',
        'realized_volatility'
    ]
}

print("Covariate Configuration:")
print("=" * 50)
for category, variables in covariate_config.items():
    if category != 'target':
        print(f"\n{category.upper()} ({len(variables)} variables):")
        for var in variables:
            print(f"  - {var}")

total_features = (len(covariate_config['future_known']) + 
                 len(covariate_config['exogenous']) + 
                 len(covariate_config['lagged']))
print(f"\nTotal input features: {total_features}")

## 5. Create Dataset with Proper Covariate Handling

Key insight: Different covariates can be used differently during training vs. prediction

In [ ]:
class FinancialTimeSeriesDataset(Dataset):
    """
    Dataset that properly handles different covariate types
    
    During training:
    - Use all covariates (we have historical data)
    
    During prediction:
    - Future-known: Can use actual future values
    - Exogenous: Need forecasts or scenarios
    - Lagged: Only use historical values
    """
    def __init__(self, df, config, window_size=60, forecast_horizon=5, 
                 mode='train', scaler=None):
        """
        Args:
            df: DataFrame with all data
            config: Covariate configuration dictionary
            window_size: Number of past days to look at
            forecast_horizon: Number of days to predict ahead
            mode: 'train' or 'predict'
            scaler: Fitted StandardScaler (provide for val/test sets)
        """
        self.df = df.copy()
        self.config = config
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mode = mode
        
        # Prepare feature columns
        self.target_col = config['target']
        self.feature_cols = (config['future_known'] + 
                            config['exogenous'] + 
                            config['lagged'])
        
        # Normalize features
        if scaler is None:
            self.scaler = StandardScaler()
            self.df[self.feature_cols] = self.scaler.fit_transform(
                self.df[self.feature_cols]
            )
        else:
            self.scaler = scaler
            self.df[self.feature_cols] = self.scaler.transform(
                self.df[self.feature_cols]
            )
        
        # Normalize target separately (for easy inverse transform)
        self.target_scaler = StandardScaler()
        self.df[self.target_col] = self.target_scaler.fit_transform(
            self.df[[self.target_col]]
        )
        
    def __len__(self):
        return len(self.df) - self.window_size - self.forecast_horizon + 1
    
    def __getitem__(self, idx):
        # Input window: past observations
        start_idx = idx
        end_idx = idx + self.window_size
        
        # Extract features for the window
        X = self.df[self.feature_cols].iloc[start_idx:end_idx].values
        
        # Extract target for future period
        target_start = end_idx
        target_end = end_idx + self.forecast_horizon
        y = self.df[self.target_col].iloc[target_start:target_end].values
        
        # For prediction, also return future-known covariates
        # (we know calendar features for the prediction period)
        future_known_idx = [self.feature_cols.index(col) 
                           for col in self.config['future_known']]
        future_covariates = self.df[self.config['future_known']].iloc[
            target_start:target_end
        ].values
        
        return (
            torch.FloatTensor(X),                    # Historical features
            torch.FloatTensor(future_covariates),    # Future-known covariates
            torch.FloatTensor(y)                     # Target
        )

# Split data (70/15/15)
train_size = int(0.7 * len(df))
val_size = int(0.15 * len(df))

train_df = df.iloc[:train_size].reset_index(drop=True)
val_df = df.iloc[train_size:train_size + val_size].reset_index(drop=True)
test_df = df.iloc[train_size + val_size:].reset_index(drop=True)

print(f"Train set: {len(train_df)} days")
print(f"Validation set: {len(val_df)} days")
print(f"Test set: {len(test_df)} days")

# Create datasets
window_size = 60  # 60 trading days ≈ 3 months
forecast_horizon = 5  # Predict next 5 days (1 week)

train_dataset = FinancialTimeSeriesDataset(
    train_df, covariate_config, window_size, forecast_horizon
)

val_dataset = FinancialTimeSeriesDataset(
    val_df, covariate_config, window_size, forecast_horizon,
    scaler=train_dataset.scaler
)

test_dataset = FinancialTimeSeriesDataset(
    test_df, covariate_config, window_size, forecast_horizon,
    scaler=train_dataset.scaler
)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check shapes
X_sample, future_cov_sample, y_sample = train_dataset[0]
print(f"\nSample shapes:")
print(f"Historical features (X): {X_sample.shape} (window_size, n_features)")
print(f"Future-known covariates: {future_cov_sample.shape} (forecast_horizon, n_future_known)")
print(f"Target (y): {y_sample.shape} (forecast_horizon,)")

## 6. Model Architecture with Covariate Integration

This model architecture properly handles different covariate types

In [ ]:
class CovariateAwareLSTM(nn.Module):
    """
    LSTM that properly integrates different types of covariates
    
    Architecture:
    1. Encode historical sequence (all past covariates)
    2. Integrate future-known covariates for prediction period
    3. Generate multi-step forecast
    """
    def __init__(self, n_features, n_future_known, hidden_size=128, 
                 num_layers=2, forecast_horizon=5, dropout=0.2):
        super(CovariateAwareLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.forecast_horizon = forecast_horizon
        
        # Encoder: Process historical sequence
        self.encoder_lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        # Future covariate processor
        self.future_cov_processor = nn.Sequential(
            nn.Linear(n_future_known, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Decoder: Generate forecasts
        # Combines LSTM encoding + future covariates
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size + hidden_size // 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, 1)
        )
    
    def forward(self, x_historical, x_future_known):
        """
        Args:
            x_historical: (batch, window_size, n_features) - all past covariates
            x_future_known: (batch, forecast_horizon, n_future_known) - calendar features
        
        Returns:
            predictions: (batch, forecast_horizon) - predicted values
        """
        batch_size = x_historical.size(0)
        
        # Encode historical sequence
        lstm_out, (hidden, cell) = self.encoder_lstm(x_historical)
        
        # Use last hidden state as context
        context = lstm_out[:, -1, :]  # (batch, hidden_size)
        
        # Generate predictions for each future time step
        predictions = []
        
        for t in range(self.forecast_horizon):
            # Get future-known covariates for time t
            future_t = x_future_known[:, t, :]  # (batch, n_future_known)
            
            # Process future covariates
            future_encoded = self.future_cov_processor(future_t)
            
            # Combine context + future covariates
            combined = torch.cat([context, future_encoded], dim=1)
            
            # Predict
            pred_t = self.decoder(combined)  # (batch, 1)
            predictions.append(pred_t)
        
        # Stack predictions
        predictions = torch.cat(predictions, dim=1)  # (batch, forecast_horizon)
        
        return predictions

# Instantiate model
n_features = len(covariate_config['future_known']) + \
             len(covariate_config['exogenous']) + \
             len(covariate_config['lagged'])
n_future_known = len(covariate_config['future_known'])

model = CovariateAwareLSTM(
    n_features=n_features,
    n_future_known=n_future_known,
    hidden_size=128,
    num_layers=2,
    forecast_horizon=forecast_horizon,
    dropout=0.2
).to(device)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\nInput features breakdown:")
print(f"  - Future-known: {n_future_known}")
print(f"  - Exogenous: {len(covariate_config['exogenous'])}")
print(f"  - Lagged: {len(covariate_config['lagged'])}")
print(f"  - Total: {n_features}")

## 7. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for x_hist, x_future, y in loader:
        x_hist = x_hist.to(device)
        x_future = x_future.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        predictions = model(x_hist, x_future)
        loss = criterion(predictions, y)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for x_hist, x_future, y in loader:
            x_hist = x_hist.to(device)
            x_future = x_future.to(device)
            y = y.to(device)
            
            predictions = model(x_hist, x_future)
            loss = criterion(predictions, y)
            total_loss += loss.item()
    
    return total_loss / len(loader)

# Training configuration
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

# Training loop
num_epochs = 50
train_losses = []
val_losses = []
best_val_loss = float('inf')

print("Training Covariate-Aware LSTM...\n")

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/home/claude/best_covariate_model.pth')
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Val Loss: {val_loss:.6f}")
        print(f"  Best Val Loss: {best_val_loss:.6f}")
        print()

# Plot training history
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss', alpha=0.7, linewidth=2)
plt.plot(val_losses, label='Validation Loss', alpha=0.7, linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Training History - Covariate-Aware LSTM', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTraining complete!")
print(f"Best validation loss: {best_val_loss:.6f}")

## 8. Evaluation

In [ ]:
# Load best model and evaluate
model.load_state_dict(torch.load('/home/claude/best_covariate_model.pth'))
model.eval()

# Make predictions on test set
all_predictions = []
all_targets = []

with torch.no_grad():
    for x_hist, x_future, y in test_loader:
        x_hist = x_hist.to(device)
        x_future = x_future.to(device)
        
        predictions = model(x_hist, x_future)
        
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(y.numpy())

predictions = np.concatenate(all_predictions, axis=0)
targets = np.concatenate(all_targets, axis=0)

# Inverse transform to get actual prices
predictions_actual = test_dataset.target_scaler.inverse_transform(
    predictions.reshape(-1, 1)
).reshape(predictions.shape)

targets_actual = test_dataset.target_scaler.inverse_transform(
    targets.reshape(-1, 1)
).reshape(targets.shape)

# Calculate metrics
mse = np.mean((predictions_actual - targets_actual) ** 2)
mae = np.mean(np.abs(predictions_actual - targets_actual))
rmse = np.sqrt(mse)
mape = np.mean(np.abs((targets_actual - predictions_actual) / targets_actual)) * 100

print("Test Set Performance:")
print("=" * 50)
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

# Calculate directional accuracy
actual_direction = np.sign(targets_actual[:, 1:] - targets_actual[:, :-1])
pred_direction = np.sign(predictions_actual[:, 1:] - predictions_actual[:, :-1])
directional_accuracy = np.mean(actual_direction == pred_direction) * 100
print(f"\nDirectional Accuracy: {directional_accuracy:.2f}%")

# Visualize predictions
n_examples = 6
fig, axes = plt.subplots(n_examples, 1, figsize=(15, 12))

for i in range(n_examples):
    idx = i * (len(predictions_actual) // n_examples)
    time_steps = np.arange(forecast_horizon)
    
    axes[i].plot(time_steps, targets_actual[idx], 
                marker='o', label='Actual', linewidth=2, markersize=8)
    axes[i].plot(time_steps, predictions_actual[idx], 
                marker='s', label='Predicted', linewidth=2, markersize=8)
    axes[i].fill_between(time_steps, targets_actual[idx], 
                         predictions_actual[idx], alpha=0.2)
    
    axes[i].set_ylabel('Price ($)', fontsize=10)
    axes[i].set_title(f'Forecast Example {i+1}', fontsize=11, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Days Ahead', fontsize=11)
plt.tight_layout()
plt.show()

## 9. Key Takeaways

### What We Learned:

1. **Covariate Categories**:
   - Future-known (calendar features) → Can use for predictions
   - Exogenous (macro factors) → Need forecasts or scenarios
   - Lagged (historical values) → Only past available

2. **Model Architecture**:
   - Separate processing for different covariate types
   - Future-known information integrated at prediction time
   - Encoder-decoder structure for multi-step forecasting

3. **JPMorgan Applications**:
   - Trading: Market data as exogenous, calendar as future-known
   - Risk: Scenario testing with macro variables
   - Credit: Economic indicators as exogenous, payment dates as future-known

### Next Steps:
- Try with real data (Yahoo Finance, FRED API)
- Add attention mechanisms
- Implement probabilistic forecasting
- Build ensemble models
- Add transaction cost modeling